# Spend Analysis

This is an analysis of our annual spending patterns for 2019.

In [ ]:
import analysis

print('Loading data...')

db = analysis.create_db_connection()
    
analysis.fill_transactions(db, 'data/2019.ods')
analysis.fill_taxonomy(db, 'data/taxonomy.json')

print('Finished loading data.')

## Taxonomy Cross Check

Check to see what spend categorizations have not been inserted into the taxonomy.

In [ ]:
c = db.cursor()

data = c.execute("""
SELECT DISTINCT category
FROM transactions
WHERE category NOT IN (SELECT id FROM full_taxonomy)
""")

bad_categories = []
for r in data:
    bad_categories.append(r[0])

if len(bad_categories) > 0:
    print(bad_categories)
else:
    print('All categories are in the taxonomy.')

## Full Year Spend Categorization

In [ ]:
from pyecharts.charts import Sunburst
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

c = db.cursor()

query = """
WITH RECURSIVE
    q AS (
        SELECT
            transactions.*,
            full_taxonomy.parent AS parent,
            0 as level
        FROM transactions
        INNER JOIN full_taxonomy
        ON transactions.category = full_taxonomy.id
        UNION
        SELECT
            q.year,
            q.month,
            q.day,
            q.vendor,
            q.credit,
            q.debit,
            q.account,
            q.parent AS category,
            full_taxonomy.parent AS parent,
            q.level + 1 AS level
        FROM q
        INNER JOIN full_taxonomy
        ON q.parent = full_taxonomy.id
    )
SELECT
    t.*, taxonomy.name
FROM
    (SELECT
        q.category,
        q.parent,
        MAX(q.level) as level,
        SUM(q.credit) - SUM(q.debit) AS amount
    FROM q
    GROUP BY q.category, q.parent) AS t
INNER JOIN taxonomy
ON t.category = taxonomy.id
ORDER BY t.level DESC, t.amount DESC
"""

data = c.execute(query)

data_set = []
v = {}
for r in data:    
    item = opts.SunburstItem(
        name=r[4],
        value=r[3],
        children=[]
    )
    
    if r[1] in v:
        v[r[1]].opts["children"].append(item)
        
    v[r[0]] = item
    
    if r[1] is None:
        data_set.append(item)

c = (
        Sunburst()
        .add(series_name="", data_pair=data_set, radius=[0, "90%"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="2019 Categorized Spend"),
            tooltip_opts=opts.TooltipOpts(formatter=JsCode("""function(param) {
                return `${param.name} $${param.value.toFixed(2)}`
            }
            """))
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}'))
    )
c.render_notebook()


In [ ]:
from pyecharts.charts import Pie
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

c = db.cursor()

query = """
SELECT taxonomy.name, t.amount
FROM
    (SELECT 
        transactions.category,
        SUM(credit) - SUM(DEBIT) AS amount
    FROM transactions
    INNER JOIN taxonomy 
    ON transactions.category = taxonomy.id
    GROUP BY transactions.category) AS t
INNER JOIN taxonomy
ON taxonomy.id = t.category
ORDER BY t.amount DESC
"""

data = c.execute(query)

v = []
for r in data:
    v.append(r)

p = (
    Pie()
    .add(
        "",
        v,
        radius=["40%", "75%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="2019 Full Year Spend"),
        legend_opts=opts.LegendOpts(
            is_show=False
        ),
        tooltip_opts=opts.TooltipOpts(formatter=JsCode("""function(param) {
            return `${param.name} $${param.value.toFixed(2)}`
        }"""))
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter=JsCode("""function(param) {
        return `${param.name} $${param.value.toFixed(2)} (${param.percent}%)`
    }
    """)))
)

p.render_notebook()


## Month by Month Spend Breakdown

In [ ]:
from IPython.display import HTML
import calendar

c = db.cursor()

query = """
SELECT
    t.month,
    taxonomy.name,
    t.amount
FROM
    (SELECT 
        month,
        category,
        SUM(credit) - SUM(DEBIT) AS amount
    FROM transactions
    INNER JOIN taxonomy
    ON transactions.category = taxonomy.id
    GROUP BY category, month) AS t
INNER JOIN taxonomy
ON t.category = taxonomy.id
ORDER BY month, amount DESC
"""

data = c.execute(query)

v = {}
for r in data:
    if r[1] not in v:
       v[r[1]] = {} 
    v[r[1]][r[0]] = r[2]

html = "<table><thead><tr><td></td>"
for month in range(1,13):
    html += "<td>" + calendar.month_name[month] + "</td>"
html += "</tr></thead><tbody>"
for key, value in v.items():
    html += "<tr><td>" + key + "</td>"
    for month in range(1,13):
        html += "<td>"
        if month in value:
            html += '${:.2f}'.format(value[month])
        else:
            html += '--'
        html += "</td>"
    html += "</tr>"
    
html += "</tbody>"

query = """
SELECT 
    month,
    SUM(credit) - SUM(debit) AS amount
FROM transactions
INNER JOIN taxonomy
ON transactions.category = taxonomy.id
GROUP BY month
ORDER BY month
"""

data = c.execute(query)

html += "<tfoot><tr><td></td>"
for row in data:
    html += "<td>${:.2f}</td>".format(row[1])

html += "</tr></tfoot>"

html += "</tbody></table>"

h = HTML(html)
display(h)

## Budgeting

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker

c = db.cursor()

query = """
WITH RECURSIVE
    month_range(x) AS (
        SELECT 1
        UNION ALL
        SELECT x+1
        FROM month_range
        LIMIT 12),
    budgets AS (
        SELECT
            month_range.x AS month,
            taxonomy.id AS category,
            taxonomy.budget * month_range.x AS amount
        FROM taxonomy
        CROSS JOIN month_range
        WHERE taxonomy.budget IS NOT NULL),
    cumulative_spend AS (
        SELECT
            category_spend.month,
            category_spend.category,
            taxonomy.name,
            SUM(category_spend.amount) OVER (PARTITION BY category_spend.category ORDER BY category_spend.month DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS amount
        FROM
            (SELECT
                t.month,
                t.category,
                SUM(t.credit) - SUM(t.debit) AS amount
            FROM (
                SELECT budgets.month, budgets.category, transactions.credit, transactions.debit
                FROM budgets
                LEFT JOIN transactions
                ON budgets.month = transactions.month
                AND budgets.category = transactions.category) AS t
            GROUP BY t.month, t.category) AS category_spend
        INNER JOIN taxonomy
        ON taxonomy.id = category_spend.category
        ORDER BY category_spend.month, amount DESC)
SELECT
    cumulative_spend.month,
    cumulative_spend.category,
    cumulative_spend.name,
    budgets.amount / 100 - cumulative_spend.amount AS amount
FROM cumulative_spend
INNER JOIN budgets
ON budgets.month = cumulative_spend.month
AND budgets.category = cumulative_spend.category
WHERE cumulative_spend.month = 12
ORDER BY amount DESC
"""
data = c.execute(query)

categories = []
values = [] 
for r in data:
    categories.append(r[2])

    if r[3] < 0:
        values.append(opts.BarItem(
            name=r[2],
            value=r[3],
            itemstyle_opts=opts.ItemStyleOpts(color="#d62728"),
        ))
    else:
        values.append(opts.BarItem(
            name=r[2],
            value=r[3],
            itemstyle_opts=opts.ItemStyleOpts(color="#1f77b4"),
        ))

bar_chart = (
    Bar()
        .add_xaxis(categories)
        .add_yaxis(series_name="Category", yaxis_data=values, markline_opts=None)
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(
            color="#000000",
            position="right",
            formatter=JsCode("""function(param) {
                return `${param.name} ($${param.value.toFixed(2)})`
            }""")))
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(is_show=False),
            yaxis_opts=opts.AxisOpts(is_show=False),
            title_opts=opts.TitleOpts(title="Budget Gap"),
            legend_opts=opts.LegendOpts(is_show=False),
            tooltip_opts=opts.TooltipOpts(is_show=False))
    )

bar_chart.render_notebook()